##BÖLÜM 1: Kurulum ve Hazırlık
###1. Adım: Temel Kütüphanelerin Kurulması
Bu komut, projenin ana omurgasını oluşturan kütüphaneleri kurar.
* flask: Web arayüzünü sunmak için kullanılan web sunucusu kütüphanesidir.

* flask-ngrok: (Bu projede doğrudan kullanılmasa da) Genellikle Colab'daki sunucuları dış dünyaya açmak için kullanılır.

* google-generativeai: Google'ın Gemini gibi üretken yapay zeka modellerine erişim sağlar.

* langchain-google-genai: LangChain kütüphanesinin Google modelleriyle konuşmasını sağlayan adaptördür.

* langchain-community: LangChain'in ChromaDB gibi topluluk tarafından geliştirilen bileşenlerini içerir.

* chromadb: Vektör veritabanıdır. Tariflerimizin "anlamsal" olarak depolandığı yerdir.

* markdown: Model cevaplarını formatlamak için kullanılır.

* python-dotenv: API anahtarı gibi gizli bilgileri .env dosyalarından okumak için kullanılır.


In [ ]:
!pip install flask flask-ngrok google-generativeai langchain-google-genai langchain-community chromadb markdown python-dotenv

###2. Adım: Google Kütüphanelerini Güncelleme

Google'ın kütüphanelerinin en güncel sürümlerinin kullanılmasını sağlar.
--upgrade bayrağı, bu paketler zaten kuruluysa bile onları en son sürüme yükseltir. Bu, yeni özelliklere erişim ve hata düzeltmeleri için önemlidir.

In [ ]:
!pip install --upgrade google-generativeai langchain-google-genai google-api-python-client google-auth

  Using cached langchain_google_genai-3.0.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-1.0.0-py3-none-any.whl.metadata (3.4 kB)
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-2.1.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_google_genai-2.1.11-py3-none-any.whl.metadata (6.7 kB)
  Using cached langchain_google_genai-2.1.10-py3-none-any.whl.metadata (7.2 kB)
  Using cached langchain_google_genai-2.1.9-py3-none-any.whl.metadata (7.2 kB)
  Using cached langchain_google_genai-2.1.8-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_google_genai-2.1.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_google_genai-2.1.6-py3-none-any.whl.metadata (7.0 kB)
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requireme

###3. Adım: Pandas Kütüphanesinin Kurulması

Veri setimiz olan .csv dosyasını okumak ve işlemek için pandas kütüphanesi kurulur.
pandas, Python'da Excel veya CSV dosyaları gibi tablo verileriyle çalışmak için endüstri standardı olan bir kütüphanedir.

In [ ]:
!pip install pandas

##BÖLÜM 2: Veritabanı Oluşturma

Bu bölümde, ham turkish_food.csv verisi işlenir ve yapay zekanın anlayabileceği bir "hafıza" (vektör veritabanı) oluşturulur.

###4. Adım: Veri İşleme Betiği
(create_database.py)

Bu blok, create_database.py adında bir Python dosyası oluşturur. Bu dosya, CSV'yi okuyan, temizleyen, vektörlere dönüştüren ve ChromaDB'ye kaydeden tüm mantığı içerir.

Bu betik, projenin "Hafıza Yükleme" (Data Ingestion) aşamasıdır.

* Temizler: turkish_food.csv dosyasındaki metinleri (norm_str, norm_ingredients) ve sayıları (to_int) standart bir formata sokar.

* Hazırlar: Her tarif için build_doc_text ile bir "bilgi kartı" oluşturur.

* Vektörleştirir: GoogleGenerativeAIEmbeddings kullanarak bu bilgi kartlarının "anlamını" temsil eden matematiksel vektörler (sayı dizileri) oluşturur.

* Depolar: Bu vektörleri ve meta verileri (prep_time vb.) ChromaDB'ye kaydeder. Bu veritabanı ./chroma_db adında bir klasörde saklanır.

In [ ]:
%%writefile create_database.py
import os
import re
import sys
import csv
import shutil
import argparse
import hashlib
from typing import List, Dict, Any

import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

DB_PATH_DEFAULT = "./chroma_db"
COLLECTION_NAME_DEFAULT = "tarif_chatbot"
EMBED_MODEL_DEFAULT = "models/text-embedding-004"

REQUIRED_COLS = {
    "name", "diet", "course", "flavor_profile",
    "prep_time", "cook_time", "state", "region", "ingredients"
}

def norm_str(x: Any) -> str:
    if pd.isna(x):
        return ""
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def norm_ingredients(s: str) -> List[str]:
    if not s:
        return []
    parts = re.split(r"[;,]", s)
    clean, seen, uniq = [], set(), []
    for p in parts:
        p = p.strip().lower()
        p = re.sub(r"^[\-\•\·]\s*", "", p)
        if p:
            clean.append(p)
    for item in clean:
        if item not in seen:
            uniq.append(item); seen.add(item)
    return uniq

def to_int(x: Any) -> int:
    try:
        v = pd.to_numeric(x, errors="coerce")
        return int(v) if pd.notna(v) else 0
    except Exception:
        return 0

def build_doc_text(row: Dict[str, Any], ing_list: List[str]) -> str:
    ing_block = "\n".join(f"- {i}" for i in ing_list) if ing_list else "- (belirtilmemiş)"
    return (
        f"Tarif Adı: {row['name']}\n"
        f"Diyet Türü: {row['diet']}\n"
        f"Kategori: {row['course']}\n"
        f"Lezzet Profili: {row['flavor_profile']}\n"
        f"Hazırlama Süresi: {row['prep_time']} dk\n"
        f"Pişirme Süresi: {row['cook_time']} dk\n"
        f"Bölge: {row['state']} / {row['region']}\n\n"
        f"Malzemeler:\n{ing_block}\n\n"
        "Hazırlanışı:\n"
        "Bu veri setinde detaylı yapılış talimatı yoktur. Gerekirse model, genel adımlar önerebilir.\n"
    )

def deterministic_id(name: str, state: str, region: str) -> str:
    base = f"{name}::{state}::{region}"
    return hashlib.sha1(base.encode("utf-8")).hexdigest()

def read_csv(csv_path: str) -> pd.DataFrame:
    try:
        try:
            return pd.read_csv(csv_path, encoding="utf-8")
        except UnicodeDecodeError:
            return pd.read_csv(csv_path, encoding="utf-8-sig")
    except Exception as e:
        raise RuntimeError(f"CSV okunurken bir hata oluştu: {e}")

def validate_columns(df: pd.DataFrame):
    missing = REQUIRED_COLS - set(df.columns)
    if missing:
        raise RuntimeError(f"CSV'de eksik sütun(lar): {missing}")

def create_database(
    csv_path: str,
    db_path: str = DB_PATH_DEFAULT,
    collection_name: str = COLLECTION_NAME_DEFAULT,
    embed_model: str = EMBED_MODEL_DEFAULT,
    force_rebuild: bool = True,
    limit: int = 0
):
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("GOOGLE_API_KEY ortam değişkeninde bulunamadı.")

    print("Yemek Tarifi Veritabanı oluşturuluyor…")
    df = read_csv(csv_path)
    validate_columns(df)

    if limit and limit > 0:
        df = df.head(limit)

    df = df.fillna("").copy()
    for col in ["name", "diet", "course", "flavor_profile", "state", "region", "ingredients"]:
        df[col] = df[col].map(norm_str)
    df["prep_time"] = df["prep_time"].map(to_int)
    df["cook_time"] = df["cook_time"].map(to_int)

    texts, metadatas, ids = [], [], []

    print(f"Toplam {len(df)} tarif işleniyor…")
    for _, r in df.iterrows():
        ing_list = norm_ingredients(r["ingredients"])
        doc_text = build_doc_text(r, ing_list)
        texts.append(doc_text)

        # CHROMA METADATA: yalnızca primitive tipler
        metadatas.append({
            "name": r["name"],
            "diet": r["diet"].lower(),
            "course": r["course"].lower(),
            "flavor_profile": r["flavor_profile"].lower(),
            "state": r["state"],
            "region": r["region"],
            "prep_time_min": int(r["prep_time"]),
            "cook_time_min": int(r["cook_time"]),
            "ingredients": ", ".join(ing_list),  # liste yerine string
            "source": os.path.basename(csv_path)
        })
        ids.append(deterministic_id(r["name"], r["state"], r["region"]))

    if force_rebuild and os.path.exists(db_path):
        print("Eski veritabanı bulundu, siliniyor…")
        shutil.rmtree(db_path)

    embedding_function = GoogleGenerativeAIEmbeddings(
        model=embed_model,
        google_api_key=api_key
    )

    print("Vektör veritabanı oluşturuluyor…")
    vectordb = Chroma.from_texts(
        texts=texts,
        embedding=embedding_function,
        persist_directory=db_path,
        collection_name=collection_name,
        metadatas=metadatas,
        ids=ids
    )
    vectordb.persist()
    print(f"Veritabanı '{db_path}' içine oluşturuldu. Koleksiyon: '{collection_name}'.")

def parse_args():
    p = argparse.ArgumentParser(description="Tarif Chatbot için Chroma veritabanı oluşturucu")
    p.add_argument("--csv", type=str, default="indian_food.csv", help="CSV yolu (varsayılan: indian_food.csv)")
    p.add_argument("--db", type=str, default=DB_PATH_DEFAULT, help="Chroma persist dizini")
    p.add_argument("--collection", type=str, default=COLLECTION_NAME_DEFAULT, help="Koleksiyon adı")
    p.add_argument("--embed-model", type=str, default=EMBED_MODEL_DEFAULT, help="Embedding modeli")
    p.add_argument("--no-force", action="store_true", help="Var olan DB’yi silmeden üzerine yaz")
    p.add_argument("--limit", type=int, default=0, help="İlk N kaydı al (0=hepsi)")
    return p.parse_args()

if __name__ == "__main__":
    args = parse_args()
    try:
        create_database(
            csv_path=args.csv,
            db_path=args.db,
            collection_name=args.collection,
            embed_model=args.embed_model,
            force_rebuild=not args.no_force,
            limit=args.limit
        )
    except Exception as e:
        print(f"HATA: {e}", file=sys.stderr)
        sys.exit(1)


Writing create_database.py



###5.Adım: Veritabanını Oluşturma Komutu
Bu komut, bir önceki adımda oluşturduğumuz create_database.py betiğini çalıştırır ve ona turkish_food.csv dosyasını kaynak olarak gösterir.

Bu komutu çalıştırdığımızda, terminalde "Yemek Tarifi Veritabanı oluşturuluyor…" ve "Vektör veritabanı oluşturuluyor…" gibi çıktıları göreceğiz. İşlem bittiğinde, Colab'ın sol tarafındaki "Dosyalar" panelinde chroma_db adında yeni bir klasör oluşacaktır.

In [ ]:
!python create_database.py --csv turkish_food.csv


HATA: GOOGLE_API_KEY ortam değişkeninde bulunamadı.


##BÖLÜM 3: Web Arayüzü (Frontend)
Kullanıcının chatbot ile etkileşime gireceği web sayfasının oluşturulduğu bölümdür.

###6. Adım: templates Klasörünü Oluşturma
Flask, HTML dosyalarını varsayılan olarak templates adında bir klasörde arar. Bu komut, o klasörü oluşturur.

In [ ]:
import os
os.makedirs("templates", exist_ok=True)

###7. Adım: Web Arayüzü Dosyası (templates/index.html)
Kullanıcının gördüğü chatbot arayüzünü tanımlayan HTML, CSS ve JavaScript kodlarının tümü bu dosyaya yazılır. Bu dosya 3 kısımdan oluşur:
##HTML (Yapı):
Sayfanın iskeletini oluşturur. Sol tarafta sohbet geçmişini gösteren menü (#sidebar) ve sağdaki ana mesajlaşma ekranı (#chat-container) gibi temel yapısal bölümleri tanımlar. Ayrıca {% ... %} gibi özel etiketler içerir; bu etiketler, Flask sunucusunun sohbet geçmişi gibi dinamik verileri HTML'in içine yerleştirmesini sağlar.

##CSS (Stil):
 Style etiketleri arasında yer alan bu kodlar, sayfanın görsel tasarımını belirler. Renkler, yazı tipleri, mesaj kutularının görünümü ve elemanların ekrandaki hizalanması gibi tüm estetik detayları yönetir.

##JavaScript (İşlevsellik):
Script etiketleri içindeki bu kod, sayfayı interaktif hale getirir ve "beyin" ile (sunucuyla) iletişim kurar. En önemli görevleri şunlardır:

Kullanıcı "Gönder" butonuna bastığında veya Enter'a tıkladığında, mesaj kutusundaki metni alır.

fetch("/send_message", ...) komutunu kullanarak bu metni arka planda çalışan Flask sunucusuna (app.py) gönderir.

Sunucudan gelen (modelin ürettiği) cevabı (data.response) bekler ve bu cevabı addMessage fonksiyonu aracılığıyla sohbet penceresinde gösterir.

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="tr">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1.0" />
  <title>Yemek Tarifi Chatbot</title>
  <style>
    body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Arial, sans-serif; display:flex; height:100vh; margin:0; background:#f4f4f9; }
    #sidebar { width:260px; background:#202123; color:#fff; padding:10px; border-right:1px solid #333; overflow:auto; display:flex; flex-direction:column; }
    #sidebar a { display:block; color:#ececf1; text-decoration:none; padding:10px 15px; margin-bottom:5px; border-radius:6px; font-size:0.9em; white-space:nowrap; overflow:hidden; text-overflow:ellipsis; }
    #sidebar a:hover { background:#343541; }
    #sidebar a.active { background:#444654; color:#fff; }
    #new-chat-btn { background:#343541; color:#fff; border:1px solid #555; padding:10px 15px; width:100%; border-radius:6px; cursor:pointer; margin-bottom:15px; text-align:left; font-size:0.9em; }
    #new-chat-btn:hover { background:#444654; }
    #conversation-list { flex-grow:1; }
    #chat-container { flex-grow:1; display:flex; flex-direction:column; background:#343541; }
    #chat-window { flex-grow:1; padding:20px; overflow:auto; color:#ececf1; }
    .message { margin-bottom:20px; padding:15px 20px; border-radius:12px; max-width:85%; line-height:1.6; }
    .user { background:#007bff; color:#fff; margin-left:auto; }
    .bot { background:#444654; color:#d1d5db; margin-right:auto; }
    .message pre { white-space:pre-wrap; word-wrap:break-word; font-family:inherit; margin:0; }
    #input-area { display:flex; padding:20px; border-top:1px solid #444; background:#40414f; }
    #input-area input { flex-grow:1; padding:12px 15px; border:1px solid #555; border-radius:20px; outline:none; font-size:1em; background:#555; color:#fff; }
    #input-area button { background:#007bff; color:#fff; border:none; padding:0 20px; margin-left:10px; border-radius:20px; cursor:pointer; font-size:1.1em; }
    #input-area button:disabled { background:#555; }
  </style>
</head>
<body>
  <div id="sidebar">
    <button id="new-chat-btn">Yeni Sohbet +</button>
    <div id="conversation-list">
      {% for conv in conversations %}
      <a href="{{ url_for('load_conversation', session_id=conv.id) }}" class="{{ 'active' if conv.id == current_session else '' }}">
        {{ conv.title }}
      </a>
      {% endfor %}
    </div>
  </div>

  <div id="chat-container">
    <div id="chat-window">
      {% if not conversation_history %}
      <div style="text-align:center; margin-top:20%; color:#888;">
        <h2>Yemek Tarifi Chatbot</h2>
        <p>Örn: “Air fryer’da patates var mı?”</p>
      </div>
      {% endif %}
      {% for message in conversation_history %}
      <div class="message {{ 'user' if message.role == 'user' else 'bot' }}">
        <pre>{{ message.content }}</pre>
      </div>
      {% endfor %}
      <div id="loading-indicator" style="display:none;">
        <div class="message bot"><pre>Yazıyor...</pre></div>
      </div>
    </div>

    <form id="input-area">
      <input type="text" id="message-input" placeholder="Bir mesaj yazın..." autocomplete="off" />
      <button type="submit" id="send-button">Gönder</button>
    </form>
  </div>

  <script>
    const chatWindow = document.getElementById("chat-window");
    const loadingIndicator = document.getElementById("loading-indicator");
    const sendButton = document.getElementById("send-button");
    const input = document.getElementById("message-input");

    document.getElementById("input-area").addEventListener("submit", async (e) => {
      e.preventDefault();
      const message = input.value.trim();
      if (!message) return;

      addMessage(message, "user");
      input.value = "";
      sendButton.disabled = true;
      loadingIndicator.style.display = "block";

      try {
        const response = await fetch("/send_message", {
          method: "POST",
          headers: { "Content-Type": "application/json" },
          body: JSON.stringify({ message })
        });

        if (!response.ok) {
          throw new Error(`HTTP error! status: ${response.status}`);
        }

        const data = await response.json();
        addMessage(data.response, "bot");
        updateConversationList(data.conversations);
      } catch (err) {
        console.error(err);
        addMessage(`Bir hata oluştu: ${err.message}`, "bot");
      } finally {
        sendButton.disabled = false;
        loadingIndicator.style.display = "none";
        input.focus();
      }
    });

    document.getElementById("new-chat-btn").addEventListener("click", async () => {
      try {
        const response = await fetch("/new_chat", { method: "POST" });
        const data = await response.json();
        if (data.success) {
          window.location.href = `/conversation/${data.new_session_id}`;
        }
      } catch (err) {
        console.error("Yeni sohbet hatası:", err);
      }
    });

    function addMessage(content, role) {
      const welcomeMsg = chatWindow.querySelector('div[style*="text-align: center"]');
      if (welcomeMsg) welcomeMsg.remove();

      const msgDiv = document.createElement("div");
      msgDiv.className = `message ${role}`;

      const pre = document.createElement("pre");
      pre.textContent = content;
      msgDiv.appendChild(pre);

      chatWindow.insertBefore(msgDiv, loadingIndicator);
      chatWindow.scrollTop = chatWindow.scrollHeight;
    }

    function updateConversationList(conversations) {
      const listDiv = document.getElementById("conversation-list");
      const currentSessionId = "{{ current_session }}";
      listDiv.innerHTML = "";

      conversations.forEach((conv) => {
        const a = document.createElement("a");
        a.href = `/conversation/${conv.id}`;
        a.textContent = conv.title;
        if (conv.id === currentSessionId) a.className = "active";
        listDiv.appendChild(a);
      });
    }

    window.onload = () => {
      chatWindow.scrollTop = chatWindow.scrollHeight;
    };
  </script>
</body>
</html>


Writing templates/index.html


##BÖLÜM 4: Sunucu Uygulaması (Backend)
Bu bölüm, projenin "beyni" olan Flask sunucusunu oluşturur. Kullanıcıdan gelen istekleri alır, RAG sürecini işletir ve yanıtı geri gönderir.

###8. Adım: Flask Sunucu Uygulaması (app.py)
Bu blok, app.py adında ana sunucu dosyasını oluşturur. Tüm RAG mantığı, veritabanı bağlantısı ve modelin çağrılması burada gerçekleşir.
Bu dosya projenin beynidir:

Başlangıç: Sunucu (app) ve Gemini modeli (model) yüklenir.

Hafıza: load_database() fonksiyonu, Bölüm 2'de oluşturulan chroma_db veritabanını yükler.

RAG Çekirdeği (get_answer): Bu fonksiyon RAG işlemini yapar:

* Retrieval: vectordb.similarity_search ile veritabanından ilgili tarifleri getirir.

* Augmentation: Bu tarifleri bir "BAĞLAM" metni olarak birleştirir.

* Generation: Gemini modeline bu bağlamı ve kullanıcının sorusunu vererek "BAĞLAMA GÖRE" bir cevap üretmesini ister.

Web Arayüzü İletişimi (Routes):

* @app.route("/"): index.html dosyasını kullanıcıya gösterir.

* @app.route("/send_message"): index.html'den gelen mesajları alır, get_answer'a yollar ve cevabı geri döndürür.

In [ ]:
%%writefile app.py
import os
import sys
import uuid
from datetime import datetime
import shutil

from flask import Flask, jsonify, redirect, render_template, request, session, url_for

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
# Tercihen yeni paket:
# !pip -q install -U langchain-chroma
# from langchain_chroma import Chroma
from langchain_community.vectorstores import Chroma

from create_database import create_database

app = Flask(__name__, template_folder="templates")
app.secret_key = os.urandom(24)
app.config["SESSION_PERMANENT"] = False

# DEBUG AÇIK — logları görmek için
app.config["DEBUG"] = True

api_key = os.environ.get("GOOGLE_API_KEY")
if not api_key:
    print("HATA: GOOGLE_API_KEY ortam değişkeninde yok.")
    sys.exit(1)

try:
    model = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.3,
        google_api_key=api_key
    )
    print("Gemini modeli yüklendi (gemini-1.5-flash).")
except Exception as e:
    print(f"Model başlatılırken hata: {e}")
    sys.exit(1)

def load_database():
    embedding_function = GoogleGenerativeAIEmbeddings(
        model="models/text-embedding-004",
        google_api_key=api_key
    )
    db_path = "./chroma_db"
    collection_name = "tarif_chatbot"

    if not os.path.exists(db_path):
        print("Veritabanı yok, oluşturuluyor…")
        create_database("turkish_food.csv")


    try:
        vectordb = Chroma(
            persist_directory=db_path,
            embedding_function=embedding_function,
            collection_name=collection_name,
        )
        print("Vektör veritabanı yüklendi.")
        return vectordb
    except Exception as e:
        print(f"Vektör DB hatası: {e}. Yeniden oluşturuluyor…")
        if os.path.exists(db_path):
            shutil.rmtree(db_path)
        create_database("indian_food.csv")
        vectordb = Chroma(
            persist_directory=db_path,
            embedding_function=embedding_function,
            collection_name=collection_name,
        )
        print("Vektör veritabanı yeniden oluşturuldu.")
        return vectordb

vectordb = load_database()
conversations = {}

def get_answer(query, vectordb, top_k=5, max_ctx_chars=12000):
    try:
        retrieval_results = vectordb.similarity_search(query, k=top_k)
    except Exception as e:
        return f"Arama sırasında hata: {e}"

    if not retrieval_results:
        return "Veritabanımda bu konuya uygun bir tarif bulamadım. Birkaç anahtar kelime daha verebilir misin?"

    chunks, total = [], 0
    for doc in retrieval_results:
        txt = doc.page_content or ""
        if total + len(txt) > max_ctx_chars:
            break
        chunks.append(txt); total += len(txt)
    context = "\n\n---\n\n".join(chunks)

    prompt = f"""
Sen bir Yemek Tarifi Asistanısın. Görevin, BAĞLAM BİLGİSİ'ndeki tarifleri kullanarak kullanıcının sorularını yanıtlamak.

Kurallar:
1) Bağlamda uygun tarif varsa adı, malzemeler ve hazırlanışı adım adım ver.
2) Bağlamda yoksa uydurma; “Veritabanımda bu tarif yok.” de.
3) Sadece tariflerle ilgili soruları yanıtla.
4) Net, adım adım ve takip etmesi kolay yanıt yaz.

BAĞLAM:
{context}

KULLANICI SORUSU:
{query}

YANIT:
""".strip()

    try:
        resp = model.invoke(prompt)
        return resp.content if isinstance(resp.content, str) else str(resp.content)
    except Exception as e:
        return f"Model cevabı alınamadı: {e}"

@app.route("/health")
def health():
    return jsonify({"status": "ok"})

@app.route("/")
def index():
    if "session_id" not in session:
        sid = str(uuid.uuid4())
        session["session_id"] = sid
        conversations[sid] = {
            "id": sid,
            "title": "Yeni Tarif Sohbeti",
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "messages": [],
        }

    sid = session["session_id"]
    if sid not in conversations:
        conversations[sid] = {
            "id": sid,
            "title": "Yeni Tarif Sohbeti",
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "messages": [],
        }

    all_conversations = sorted(
        conversations.values(),
        key=lambda x: x.get("created_at", "1970-01-01 00:00:00"),
        reverse=True
    )
    # Eğer şablon bulunamazsa 500 atar — o yüzden template_folder yolunu yukarıda sabitledik.
    return render_template(
        "index.html",
        conversation_history=conversations[sid]["messages"],
        conversations=all_conversations,
        current_session=sid
    )

@app.route("/send_message", methods=["POST"])
def send_message():
    data = request.get_json(force=True)
    user_message = (data.get("message") or "").strip()
    sid = session.get("session_id")

    if not sid or sid not in conversations:
        return jsonify({"response": "Hata: Oturum bulunamadı.", "conversations": []}), 400

    if not user_message:
        return jsonify({"response": "Lütfen bir mesaj yazın.", "conversations": []}), 400

    conversations[sid]["messages"].append({"role": "user", "content": user_message})

    if len(conversations[sid]["messages"]) == 1:
        title = user_message[:30] + "..." if len(user_message) > 30 else user_message
        conversations[sid]["title"] = title or "Yeni Tarif Sohbeti"

    try:
        bot_response = get_answer(user_message, vectordb, top_k=5)
        conversations[sid]["messages"].append({"role": "bot", "content": bot_response})

        all_conversations = sorted(
            conversations.values(),
            key=lambda x: x.get("created_at", "1970-01-01 00:00:00"),
            reverse=True
        )
        return jsonify({"response": bot_response, "conversations": all_conversations})
    except Exception as e:
        err = "Üzgünüm, bir hata oluştu. Lütfen tekrar dener misin?"
        print(f"/send_message hata: {e}", file=sys.stderr)
        all_conversations = sorted(
            conversations.values(),
            key=lambda x: x.get("created_at", "1970-01-01 00:00:00"),
            reverse=True
        )
        return jsonify({"response": err, "conversations": all_conversations})

@app.route("/new_chat", methods=["POST"])
def new_chat():
    sid = str(uuid.uuid4())
    session["session_id"] = sid
    conversations[sid] = {
        "id": sid,
        "title": "Yeni Tarif Sohbeti",
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "messages": [],
    }
    return jsonify({"success": True, "new_session_id": sid})

@app.route("/conversation/<session_id>")
def load_conversation(session_id):
    if session_id in conversations:
        session["session_id"] = session_id
    return redirect(url_for("index"))

if __name__ == "__main__":
    # Colab proxy için 0.0.0.0:5000 şart
    app.run(host="0.0.0.0", port=5000)


Writing app.py


##BÖLÜM 5: Sunucuyu Çalıştırma ve Yayınlama
Bu bölümde, Colab ortamında sunucunun çalıştırılması ve dış dünyaya açılması sağlanır.

###9. ve 10. Adım: ngrok Kurulumları
Bu kütüphaneler, Colab'da çalışan sunucuyu (localhost:5000) herkese açık bir URL'ye tünellemek için kullanılır.

* Bu projede flask-ngrok doğrudan app.py içinde kullanılmıyor. Colab'ın kendi proxy (aracı sunucu) özelliği (Adım 13'te) bu işlevi görüyor. Ancak, pyngrok kurulumu bazı Colab ortamlarında genel proxy işlevleri için yararlı olabilir.

In [ ]:
!pip install pyngrok

In [ ]:
!pip -q install flask-ngrok

###11. Adım: API Anahtarını Güvenle Yükleme
Bu kod bloğu, Google API anahtarımızı Colab'ın "Secrets" (Gizli Anahtarlar) bölümünden güvenli bir şekilde okur ve programın kullanabilmesi için "ortam değişkeni" olarak ayarlar.
* Bu çok önemlidir! API anahtarımızı (API Key) asla kodun içine doğrudan yazmamalıyız. Colab'daki sol menüde bulunan Anahtar (🔑) simgesine tıklayarak GOOGLE_API_KEY adında bir gizli anahtar oluşturulmalı. Bu kod, anahtarı oradan güvenle okur.

In [ ]:
# HÜCRE 1 — API anahtarı
import os

# (Varsa) Colab Secrets'tan çekmeyi dene:
try:
    from google.colab import userdata
    key_from_secrets = userdata.get('GOOGLE_API_KEY')
except Exception:
    key_from_secrets = None

if key_from_secrets:
    os.environ['GOOGLE_API_KEY'] = key_from_secrets
    print("GOOGLE_API_KEY (secrets) yüklendi.")
else:
    # Buraya manuel yaz (tırnak içinde)
    os.environ['GOOGLE_API_KEY'] = "BURAYA_KENDI_GOOGLE_API_KEY'INI_YAZ"
    print("GOOGLE_API_KEY manuel yüklendi.")

# Kontrol
print("GOOGLE_API_KEY set mi?", "OK" if os.environ.get("GOOGLE_API_KEY") else "YOK")


GOOGLE_API_KEY manuel yüklendi.
GOOGLE_API_KEY set mi? OK


###12. Adım: Sunucuyu Arka Planda Başlatma
Bu komut, app.py dosyasını (Flask sunucusunu) çalıştırır.
* Buradaki & işareti çok önemlidir. Komutun "arka planda" çalışmasını sağlar. Bu sayede sunucu çalışmaya devam ederken biz de Colab'da bir sonraki hücreyi (URL'yi alma) çalıştırabiliriz. & olmasaydı, bu hücre sunucu çalıştığı sürece kilitlenirdi.

In [ ]:
# 3) Sunucuyu arka planda başlat
get_ipython().system_raw("python app.py &")


###3. Adım: Herkese Açık URL'yi Alma
Bu kod, Google Colab'ın az önce 5000 portunda başlattığımız sunucu için oluşturduğu özel, herkese açık URL'yi alır ve ekrana yazdırır.
* çıktı olarak bize verilen ve chatbotumuzun çalıştığı link https://5000-m-s-2dgccsbfqwft2-a.us-west4-1.prod.colab.dev/

In [ ]:
from google.colab import output
print("Proxy URL:", output.eval_js("google.colab.kernel.proxyPort(5000)"))


Proxy URL: https://5000-m-s-24xcr2acuuzmy-a.asia-east1-0.prod.colab.dev
